# PPG analysis

In [1]:
import os
from paradigma.config import PPGConfig, IMUConfig, SignalQualityFeatureExtractionConfig, SignalQualityFeatureExtractionConfigAcc, SignalQualityClassificationConfig, HeartRateExtractionConfig
from paradigma.preprocessing import scan_and_sync_segments, preprocess_ppg_data
from paradigma.heart_rate.heart_rate_analysis import extract_signal_quality_features_io, extract_signal_quality_features, signal_quality_classification, estimate_heart_rate

In [2]:
# Cell has the tag 'parameters'
branch = 'heart_rate'
sensor = 'ppg'

path_to_data =  '../../../tests/data'
path_to_classifier = os.path.join(path_to_data, '0.classification', sensor)
path_to_sensor_data = os.path.join(path_to_data, '1.prepared_data')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', sensor)
path_to_quality_features = os.path.join(path_to_data, '3.extracted_features', sensor)
path_to_signal_quality = os.path.join(path_to_data, '4.predictions', sensor)
path_to_hr_estimate = os.path.join(path_to_data, '5.quantification', sensor)

## Preprocessing

In [ ]:
ppg_config = PPGConfig()
imu_config = IMUConfig()

metadatas_ppg, metadatas_imu = scan_and_sync_segments(os.path.join(path_to_sensor_data, sensor), os.path.join(path_to_sensor_data, 'imu'))
df_ppg_proc, df_acc_proc = preprocess_ppg_data(metadatas_ppg[0], metadatas_imu[0], path_to_preprocessed_data, ppg_config, imu_config)

## Extract signal quality features

In [ ]:
config_ppg = SignalQualityFeatureExtractionConfig()
config_acc = SignalQualityFeatureExtractionConfigAcc()
df_features = extract_signal_quality_features(config_ppg, df_ppg_proc, config_acc, df_acc_proc)

## Signal quality classification

In [ ]:
config = SignalQualityClassificationConfig()
df_sqa = signal_quality_classification(df_features, config, path_to_classifier)

## Heart rate estimation

In [ ]:
config = HeartRateExtractionConfig()
df_hr = estimate_heart_rate(df_sqa, df_ppg_proc, config)